This notebook demonstrates some the coordinate manipulation methods available through the Simulations stack.  It will not show all of the available routines.  To investigate for yourself the methods that are available, see
    
    $SIMS_UTILS_DIR/python/lsst/sims/utils/AstrometryUtils.py
    
    $SIMS_COORDUTILS_DIR/python/lsst/sims/coordUtils/CameraUtils.py

    $SIMS_UTILS_DIR/python/lsst/sims/utils/coordinateTransformations.py

There is, unfortunately, still a little bit of redundancy between the methods offered in these two files.  Hopefully, that will get cleaned up in the near future.  All of the code uses PALPY as its backend, so it should not matter which methods you use to do a given calculation.  PALPY source code is available at

https://github.com/Starlink/palpy

You should begin by loading the stack and setting up `sims_coordUtils` using

    setup sims_coordUtils -t sims

We will start with the highest level routines: taking (RA, Dec) pairs and determining which chip they land on.  To do this, we must load a simulated camera.  The code below will load the current map of the LSST camera.

In [ ]:
from lsst.obs.lsstSim import LsstSimMapper
camera = LsstSimMapper().camera

The routines that connect (RA, Dec) on the sky with positions on the camera require information about how the telescope is pointed.  As with nearly all CatSim applications, this data is stored in the `ObservationMetaData` class.  Below, we will use the `ObservationMetaDataGenerator` introduced in the notebooks `generateAgnCatalog_150409.ipynb` and `CatSimTutorial_SimulationsAHM_1503.ipynb` to create a self-consistent `ObservationMetaData` instantation from an example OpSim run.

In [ ]:
import os
import eups
from lsst.sims.catUtils.utils import ObservationMetaDataGenerator

#the code below just points to an OpSim output database that
#is carried around with the Simulations stack for testing purposes
opSimDbName = 'opsimblitz1_1133_sqlite.db'
fullName = os.path.join(eups.productDir('sims_data'),'OpSimData',opSimDbName)

obsMD_generator = ObservationMetaDataGenerator(database=fullName, driver='sqlite')

Create an `ObservationMetaData` instantiation based on a pointing with 24 < RA < 100 (in degrees)

In [ ]:
boundLength=3.0 #the radius of our field of view in degrees
obs_metadata = obsMD_generator.getObservationMetaData(fieldRA=(24.0,100.0),
                                                      limit=1, boundLength=boundLength)
print obs_metadata[0].pointingRA, obs_metadata[0].rotSkyPos

Now we will generate a sample of 10 (RA, Dec) pairs that are within our field of view.

In [ ]:
import numpy
epoch = 2000.0
nsamples = 10
numpy.random.seed(42)
radius = boundLength*numpy.random.sample(nsamples)
theta = 2.0*numpy.pi*numpy.random.sample(nsamples)

raRaw = obs_metadata[0].pointingRA + radius*numpy.cos(theta)
decRaw = obs_metadata[0].pointingDec + radius*numpy.sin(theta)

To find what chip they fall on (given an `ObservationMetaData`), we the method `chipNameFromRaDec`.

In [ ]:
from lsst.sims.coordUtils import chipNameFromRaDec

chipNames = chipNameFromRaDec(ra=raRaw, dec=decRaw,
                              camera=camera, epoch=epoch,
                              obs_metadata=obs_metadata[0])

print chipNames


<b> Note: currently, chipNameFromRaDec only returns the names of science chips (as opposed to wavefront sensors or guide chips) on which the object falls.  It also does not know how to handle objects that fall on two chips at once.</b>

There is also a method to find the pixel coordinates on the chip of each object.

In [ ]:
from lsst.sims.coordUtils import pixelCoordsFromRaDec

pixelCoords = pixelCoordsFromRaDec(ra=raRaw, dec=decRaw,
                                   camera=camera, epoch=epoch,
                                   obs_metadata=obs_metadata[0])

for name, x, y in zip(chipNames, pixelCoords[0], pixelCoords[1]):
    print name, x, y

And methods to calculate the pupil coordinates of an object in radians.

In [ ]:
from lsst.sims.utils import pupilCoordsFromRaDec

help(pupilCoordsFromRaDec)

In [ ]:
xPup, yPup = pupilCoordsFromRaDec(raRaw, decRaw,
                                  obs_metadata=obs_metadata[0], epoch=epoch)

for x,y in zip(xPup, yPup):
    print x, y

There are also methods to transform from the International Celestial Reference System to 'observed' RA/Dec

* `observedFromICRS` applied precession, nutation, proper motion, parallax, radial velocity, annual aberration, diurnal aberration, and refraction.  It relies on the methods below.

* `appGeoFromICRS` calculates the apparent geocentric position of the object.  It applies precession, nutation, proper motion, parallax, radial velocity, and annual aberration


* `observedFromAppGeo` converts the apparent geocentric position to the observed position, adding diurnal aberration and refraction to the list of applied effects.  You will generally only want to call `observedFromAppGeo` on coordinates that have already been corrected with `appGeoFromICRS`.  This is what `observedFromICRS` does.

In [ ]:
from lsst.sims.utils import observedFromICRS

help(observedFromICRS)

In [ ]:
from lsst.sims.utils import appGeoFromICRS

help(appGeoFromICRS)

In [ ]:
from lsst.sims.utils import observedFromAppGeo

help(observedFromAppGeo)

In `generateAgnCatalog_150409.ipynb` and `CatSimTutorial_SimulationsAHM_1503.ipynb` we introduced the idea of mixins and getter methods that allow you to seamlessly incorporate calculated quantities into simulated catalogs.  `Astrometry.py` defines getter methods that allow you to incorporate the above calculations into catalogs.

* Getters to correct the (RA, Dec) coordinates of stars are provided by the mixin `AstrometryStars`.


* Getters to correct the (Ra, Dec) coordinates of galaxies are provided by the mixin `AstrometryGalaxies` (this is different from `AstrometryStars` in that `AstrometryGalaxies` knows not to bother looking for proper motion, parallax, or radial velocity)


* Getters associated with camera-based quantities are provided by the mixin `CameraCoords`.

Note: `AstrometryStars`, `AstrometryGalaxies`, and `CameraCoords` all inherit from the mixin `AstrometryBase` which provides getters for quantities and methods that are agnostic to the star/galaxy distinction

In [ ]:
from lsst.sims.catUtils.mixins import AstrometryBase
for methodName in dir(AstrometryBase):
    if 'get_' in methodName:
        print methodName

In [ ]:
from lsst.sims.catUtils.mixins import AstrometryStars
for methodName in dir(AstrometryStars):
    if 'get_' in methodName and methodName not in dir(AstrometryBase):
        print methodName

In [ ]:
from lsst.sims.catUtils.mixins import AstrometryGalaxies
for methodName in dir(AstrometryGalaxies):
    if 'get_' in methodName and methodName not in dir(AstrometryBase):
        print methodName

In [ ]:
from lsst.sims.catUtils.mixins import CameraCoords
for methodName in dir(CameraCoords):
    if 'get_' in methodName and methodName not in dir(AstrometryBase):
        print methodName

Here we illustrate how to use these mixins to include coordinate transformations into simulated catalogs.

In [ ]:
from lsst.sims.catalogs.measures.instance import InstanceCatalog
from lsst.sims.catUtils.mixins import AstrometryStars

class chipNameCatalog(InstanceCatalog, AstrometryStars, CameraCoords):
    column_outputs = ['raJ2000', 'decJ2000', 'raObserved', 'decObserved', 
                      'chipName', 'xPix', 'yPix']

    transformations = {'raJ2000':numpy.degrees, 'decJ2000':numpy.degrees,
                       'raObserved':numpy.degrees, 'decObserved':numpy.degrees}
    
    camera = LsstSimMapper().camera


In [ ]:
from lsst.sims.catUtils.baseCatalogModels import WdStarObj

#define a smaller ObservationMetaData so that we don't create an over large catalog
obs_metadata = obsMD_generator.getObservationMetaData(fieldRA=(24.0, 100.0),
                                                      limit=1, boundLength=0.5)

#again, use the white dwarf database table so that we don't get too many objects
#in this small example
starDB = WdStarObj()

testCat = chipNameCatalog(starDB, obs_metadata=obs_metadata[0])

catName = 'test_cat.txt'

if os.path.exists(catName):
    os.unlink(catName)
    
testCat.write_catalog(catName)

!cat test_cat.txt